In [1]:

import random
with open('emails.csv', 'r') as r, open('shuffled_emails_spam_ham.csv', 'w') as w:
    data = r.readlines()
    header, rows = data[0], data[1:]
    random.shuffle(rows)
    rows = '\n'.join([row.strip() for row in rows])
    w.write(header + rows)


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

os.getcwd()
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

'C:\\Users\\Akash'

### reading data

In [3]:
train_data = pd.read_csv("emails.csv")
train_data.head()

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,""" naturally irresistible your corporate identi...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,""" the stock trading gunslinger fanny is merri...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,""" unbelievable new homes made easy im wanting...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,""" 4 color printing special request additional...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,""" do not have money , get software cds from he...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



# creating a vocablary

In [4]:
def make_vocab(sentences):
    '''Build a vocabulary based on the available text data. Returns a dictionary with word and its frequency.'''
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [5]:
sentences =  train_data['text'].apply(lambda x: x.split()).values
vocab = make_vocab(sentences)


# getting word embeddings

In [6]:
import gensim
embeddings = gensim.models.KeyedVectors.load_word2vec_format('D:\Dataset\word2vec-GoogleNews-vectors-master\GoogleNews-vectors-negative300.bin',binary=True,limit=600000)

C:\Users\Akash\Anaconda3\envs\gpu_use\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Akash\Anaconda3\envs\gpu_use\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
# Evaluating how word2vec works.
#embeddings.most_similar('chelsea')
# Since this embedding was derived using Google news data, there's a large correlation between chelsea and wenger!

In [8]:
import operator
def present_in_word2vec(vocab,embeddings):
    '''Finds the intersection between embedding and our vocab.'''
    in_embedding = {}
    not_in_embedding = {}
    in_embedding_word_count = 0
    not_in_embedding_word_count = 0
    for word in vocab:
        try:
            in_embedding[word] = embeddings[word]
            in_embedding_word_count += vocab[word]
        except:
            not_in_embedding[word] = vocab[word]
            not_in_embedding_word_count += vocab[word]
            pass
    
    print('Found embeddings for {:.2%} of vocab'.format(len(in_embedding)/len(vocab)))
    print('Found embeddings for {:.2%} of all text'.format(in_embedding_word_count/(in_embedding_word_count+not_in_embedding_word_count)))
    x = sorted(not_in_embedding.items(),key=operator.itemgetter(1))[::-1]
    return x

In [9]:
out_of_embed = present_in_word2vec(vocab,embeddings) 

Found embeddings for 48.43% of vocab
Found embeddings for 62.88% of all text


In [10]:
out_of_embed[:10]

[('-', 104985),
 ('.', 91845),
 (',', 60311),
 (':', 42312),
 ('to', 41189),
 ('/', 40210),
 ('and', 27260),
 ('of', 23354),
 ('a', 19650),
 ('"', 14536)]

# data cleaning

In [11]:
import string
p = string.punctuation
l = []
for y in p:
    if y not in ["'","-","/","&"]:
        l.append(y)
        
x_ = ''.join([x for x in l])
print(x_)
def clean_punctuation(token):
    '''Cleans text by adjusting punctuations'''   
    x = str(token)
    for punct in "-/":
        x = x.replace(punct,' ')
    for punct in '&':
        # & is present in the embedding hence adding spaces around it.
        x = x.replace(punct,f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct,'')
    
    return x
        

!"#$%()*+,.:;<=>?@[\]^_`{|}~


In [12]:
train_data['text'] = train_data['text'].apply(lambda x:clean_punctuation(x))
#print(train_data['text'])
#print(type(train_data['text'])
sentences = train_data['text'].apply(lambda x: x.split()).values
print(sentences)
vocab = make_vocab(sentences)
#print(vocab)
out_of_embed = present_in_word2vec(vocab,embeddings)

[list(['naturally', 'irresistible', 'your', 'corporate', 'identity', 'lt', 'is', 'really', 'hard', 'to', 'recollect', 'a', 'company', 'the', 'market', 'is', 'full', 'of', 'suqgestions', 'and', 'the', 'information', 'isoverwhelminq', 'but', 'a', 'good', 'catchy', 'logo', 'stylish', 'statlonery', 'and', 'outstanding', 'website', 'will', 'make', 'the', 'task', 'much', 'easier', 'we', 'do', 'not', 'promise', 'that', 'havinq', 'ordered', 'a', 'iogo', 'your', 'company', 'will', 'automaticaily', 'become', 'a', 'world', 'ieader', 'it', 'isguite', 'ciear', 'that', 'without', 'good', 'products', 'effective', 'business', 'organization', 'and', 'practicable', 'aim', 'it', 'will', 'be', 'hotat', 'nowadays', 'market', 'but', 'we', 'do', 'promise', 'that', 'your', 'marketing', 'efforts', 'will', 'become', 'much', 'more', 'effective', 'here', 'is', 'the', 'list', 'of', 'clear', 'benefits', 'creativeness', 'hand', 'made', 'original', 'logos', 'specially', 'done', 'to', 'reflect', 'your', 'distinctive',

Found embeddings for 48.50% of vocab
Found embeddings for 79.43% of all text


In [13]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '', x)
    x = re.sub('[0-9]{4}', '', x)
    x = re.sub('[0-9]{3}', '', x)
    x = re.sub('[0-9]{2}', '', x)
    return x

In [14]:
train_data["text"] = train_data["text"].apply(lambda x: clean_numbers(x))
sentences = train_data["text"].apply(lambda x: x.split())
vocab = make_vocab(sentences)
print(vocab)

{'naturally': 16, 'irresistible': 4, 'your': 9274, 'corporate': 356, 'identity': 91, 'lt': 30, 'is': 12483, 'really': 417, 'hard': 200, 'to': 41189, 'recollect': 25, 'a': 19650, 'company': 1596, 'the': 48902, 'market': 1522, 'full': 467, 'of': 23354, 'suqgestions': 14, 'and': 27260, 'information': 2285, 'isoverwhelminq': 7, 'but': 2058, 'good': 1071, 'catchy': 24, 'logo': 215, 'stylish': 15, 'statlonery': 15, 'outstanding': 96, 'website': 648, 'will': 8079, 'make': 1415, 'task': 70, 'much': 859, 'easier': 86, 'we': 8637, 'do': 2594, 'not': 4461, 'promise': 78, 'that': 9135, 'havinq': 9, 'ordered': 96, 'iogo': 26, 'automaticaily': 5, 'become': 261, 'world': 464, 'ieader': 16, 'it': 6594, 'isguite': 13, 'ciear': 9, 'without': 385, 'products': 547, 'effective': 207, 'business': 2045, 'organization': 211, 'practicable': 24, 'aim': 44, 'be': 9186, 'hotat': 23, 'nowadays': 25, 'marketing': 420, 'efforts': 227, 'more': 2133, 'here': 1774, 'list': 1055, 'clear': 167, 'benefits': 137, 'creative

In [15]:
''''
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
ps = PorterStemmer()
train_data['text']=train_data['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))
'''

"'\nimport re\nimport nltk\nnltk.download('stopwords')\nfrom nltk.corpus import stopwords\nfrom nltk.stem.porter import PorterStemmer\nfrom sklearn.feature_extraction.text import CountVectorizer\nps = PorterStemmer()\ntrain_data['text']=train_data['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))\n"

In [16]:
out_of_embed = present_in_word2vec(vocab,embeddings)

Found embeddings for 53.70% of vocab
Found embeddings for 83.76% of all text


In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


# getting and cleaning test data

In [18]:
# Preparing test data
#test_data = pd.read_csv('test.csv')
#test_data['question_text'] = test_data['question_text'].fillna("_na_").values
#test_data['question_text'] = test_data['question_text'].apply(lambda x:clean_punctuation(x))
#test_data['question_text'] = test_data['question_text'].apply(lambda x:clean_numbers(x))
#test_sent = test_data['question_text'].apply(lambda x:x.split())
#test_vocab = make_vocab(test_sent)
print(train_data)

                                                   text spam Unnamed: 2  \
0      naturally irresistible your corporate identit...    1        NaN   
1      the stock trading gunslinger  fanny is merril...    1        NaN   
2      unbelievable new homes made easy  im wanting ...    1        NaN   
3      4 color printing special  request additional ...    1        NaN   
4      do not have money  get software cds from here...    1        NaN   
5      great nnews  hello  welcome to medzonline sh ...    1        NaN   
6      here  s a hot play in motion  homeland securi...    1        NaN   
7      save your money buy getting this thing here  ...    1        NaN   
8      undeliverable  home based business for grownu...    1        NaN   
9      save your money buy getting this thing here  ...    1        NaN   
10     las vegas high rise boom  las vegas is fast b...    1        NaN   
11     save your money buy getting this thing here  ...    1        NaN   
12     brighten those tee

# 1) test train val split
# 2)max seq len   = 80
# 3) max words = 20000'
# 4) tokenizing
# 5)padding the mat


In [19]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#tokenizer.fit_on_texts(list(train_data))
#train_data = tokenizer.texts_to_sequences(train_data)
#train_data = pad_sequences(train_data)
#print(len(train_data[1]))
 
train_set,test_set = train_test_split(train_data,test_size=0.2)
train_set, val_set = train_test_split(train_set,test_size= 0.2)

max_features = 20000 #no of words
maxlen = 80

tokenizer = Tokenizer(num_words=max_features)

train_X = train_set['text'].fillna("_na_").values
val_X = val_set['text'].fillna("_na_").values
test_X = test_set['text'].fillna("_na_").values

tokenizer.fit_on_texts(list(train_X))
tokenizer.fit_on_texts(list(val_X))
tokenizer.fit_on_texts(list(test_X))

train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)
#print(len(val_X[1]))

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)


train_y = train_set['spam'].values
val_y = val_set['spam'].values
test_y = test_set['spam'].values
word_index = tokenizer.word_index
print(len(word_index))

33644


In [20]:
#tokenizer.word_index.items()
import gc

# creating mebedding matrix

In [21]:
embeddings_index = {}
embedding_size = 300
for word in  embeddings.wv.vocab:
    embeddings_index[word] = embeddings.word_vec(word)

all_embeddings = np.stack(list(embeddings_index.values()))
embed_mean,embed_std = all_embeddings.mean(),all_embeddings.std()
num_words = len(tokenizer.word_index)

print(num_words)

C:\Users\Akash\Anaconda3\envs\gpu_use\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


33644


In [22]:
#print(embeddings.wv.vocab)

In [23]:
embedding_matrix = np.random.normal(embed_mean,embed_std,(num_words,embedding_size))
#print(embedding_matrix)
for word,index in tokenizer.word_index.items():
   # print(index)
    index -= 1
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
print(embedding_matrix.shape)
del(embeddings_index)
gc.collect()

(33644, 300)


0

In [24]:
from keras.layers import Dense,Input,Conv2D,Embedding,Dropout,Activation,MaxPool2D,Concatenate,Flatten,Reshape
from keras.models import Model
from keras.optimizers import Adam

inputs = Input(shape=(maxlen,))
word2vec_embedding = Embedding(len(tokenizer.word_index),300,weights = [embedding_matrix],trainable=False)(inputs)
reshape = Reshape((maxlen,300,1))(word2vec_embedding)

conv_window_3 = Conv2D(512,kernel_size=(3,300),padding='valid',activation='relu')(reshape)
conv_window_4 = Conv2D(512,kernel_size=(4,300),padding='valid',activation='relu')(reshape)
conv_window_5 = Conv2D(512,kernel_size=(5,300),padding='valid',activation='relu')(reshape)

max_pool_window_3 = MaxPool2D(pool_size=(80-3+1,1),strides=(1,1),padding='valid')(conv_window_3)
max_pool_window_4 = MaxPool2D(pool_size=(80-4+1,1),strides=(1,1),padding='valid')(conv_window_4)
max_pool_window_5 = MaxPool2D(pool_size=(80-5+1,1),strides=(1,1),padding='valid')(conv_window_5)

concat_feature_map = Concatenate(axis=1)([max_pool_window_3,max_pool_window_4,max_pool_window_5])
flatten = Flatten()(concat_feature_map)
dropout = Dropout(0.5)(flatten)

output = Dense(units=1,activation='sigmoid')(dropout)

model = Model(inputs=inputs,outputs=output)

adam = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_X,train_y, batch_size=30, epochs=2, verbose=1, validation_data=(val_X, val_y))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3664 samples, validate on 917 samples
Epoch 1/2
3664/3664 [==============================] - 11s 3ms/step - loss: 0.3583 - acc: 0.8510 - val_loss: 0.2007 - val_acc: 0.9357
Epoch 2/2
3664/3664 [==============================] - 7s 2ms/step - loss: 0.1445 - acc: 0.9465 - val_loss: 0.1455 - val_acc: 0.9509


In [25]:
from keras.models import load_model
model.save('QIQC_model.hd5')

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 80, 300)      10093200    input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 80, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 78, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [27]:
from sklearn.metrics import classification_report
print(test_X.shape)
pred = model.predict(test_X)
print(pred)
#print(classification_report(test_set,pred))


(1146, 80)
[[0.01419899]
 [0.0998497 ]
 [0.03436112]
 ...
 [0.12279618]
 [0.01687597]
 [0.9878275 ]]


In [28]:
#sub = test_set[['qid']]
prediction = np.round(np.argmax(pred,axis=1)).astype(int)
test_set['prediction'] = prediction
test_set.to_csv('submission.csv',index=False)

C:\Users\Akash\Anaconda3\envs\gpu_use\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
test_set.head()

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,prediction
3116,australian energy dear vince dzien dobry ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1387,alliance ferc alert regional market reports...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2760,re aiesec biliana i have prior commitment...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2199,re seminar series mug barbara i think you ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1186,professional logo for you now working on you...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [30]:
email = "Subject london 98,research intranet felipe following our discussion please close down all intranet access to the london  research group site  inaccuracies have been found in the content which may  have implications for our trading operations  closing the site will force  traders to contact our group direct  ensuring only correct information is  passed   the site will be completely overhauled over the coming months  so i see  little benefit in reinstating access to the site in the near future   many thanks   steve"
email = clean_punctuation(email)
email = clean_numbers(email)
print(email)
email_x = email.split()
email_vocab = make_vocab(email_x)
email_out_of_embedding = present_in_word2vec(email_vocab,embedding)
print(email_out_of_embedding)
#print(email_x)
#print(len(email_x))
email_tokenizer = Tokenizer(num_words=max_features)
email_tokenizer.fit_on_texts(list(email_x)) #tokenize the text
email_x = email_tokenizer.texts_to_sequences(email_x) #convert the text to sequence
email_x = pad_sequences(email_x)
email_index = email_tokenizer.word_index
print(len(email_index))
all_embeddings_index = {}
email_embedding_size = 300
for all_word in  embeddings.wv.vocab:
    all_embeddings_index[all_word] = embeddings.word_vec(all_word)# create a matrix of word embeddings

all_embeddings = np.stack(list(all_embeddings_index.values()))
#print(email_all_embeddings.len)
email_embed_mean,email_embed_std = all_embeddings.mean(),all_embeddings.std()
email_num_words = len(email_tokenizer.word_index)
#print(email_num_words)
email_embedding_matrix = np.random.normal(email_embed_mean,email_embed_std,(email_num_words,email_embedding_size))
#print(embedding_matrix)
for email_word,email_index in email_tokenizer.word_index.items():
   # print(index)
    email_index -= 1
    email_embedding_vector = all_embeddings_index.get(word)
    if email_embedding_vector is not None:
        email_embedding_matrix[index] = email_embedding_vector

del(all_embeddings_index)
print(email_embedding_matrix.shape)
email_embedding_matrix = email_embedding_matrix.reshape(80,0)
model.predict(email_embedding_matrix)


Subject london research intranet felipe following our discussion please close down all intranet access to the london  research group site  inaccuracies have been found in the content which may  have implications for our trading operations  closing the site will force  traders to contact our group direct  ensuring only correct information is  passed   the site will be completely overhauled over the coming months  so i see  little benefit in reinstating access to the site in the near future   many thanks   steve


NameError: name 'embedding' is not defined

In [ ]:
model_json = model.to_json()
with open("QIQC_NLP_CNN.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("QIQC_NLP_CNN.h5")
print("Saved model to disk")